In [143]:
from qiskit.circuit.library import IGate, XGate, YGate, RZGate, RYGate, RXGate
from qiskit import QuantumCircuit, execute
from numpy import pi, floor
from random import choice
from quantuminspire.qiskit import QI
from qiskit.visualization import plot_histogram

QI.set_authentication()

In [144]:
def s1(starmon):
  starmon.cnot(0,2)
  starmon.cnot(1,2)
    
def s2(starmon):
    starmon.cnot(1,2)
    starmon.cnot(3,2)
    

In [145]:
#create a random set of native gates
gateSet = [IGate(), XGate(), YGate(), RZGate(pi/4), RYGate(pi/4), RXGate(pi/4)]
gateDepth = 2
qubits = 3
gateList = []
for i in range((int)(gateDepth/2)):
    gateList.append([])
    for j in range(qubits):
        gateList[i].append(choice(gateSet))
        
#0 is encoded in the bit-flip code (|000> - do nothing)
starmon = QuantumCircuit(5,5)

#apply the random gates
for i in range((int)(gateDepth/2)):
    for j in range(qubits):
        r = j
        if j >= 2:
            r = j+1
        starmon.append(gateList[i][j],[r])
        
#apply the inverse of random gates
for i in range((int)(gateDepth/2)-1,-1,-1):
    for j in range(qubits):
        r=j
        if j >= 2:
            r= j+1
        starmon.append(gateList[i][j].inverse(), [r])
        
s1(starmon)
starmon.swap(2,4)
s2(starmon)

starmon.measure_all()

In [146]:
starmon.draw()

┌───┐      ┌───┐                            ░ ┌─┐            
   q_0: ───┤ X ├──────┤ X ├──────■─────────────────────░─┤M├────────────
        ┌──┴───┴──┐┌──┴───┴───┐  │                     ░ └╥┘┌─┐         
   q_1: ┤ Ry(π/4) ├┤ Ry(-π/4) ├──┼────■───────■────────░──╫─┤M├─────────
        └─────────┘└──────────┘┌─┴─┐┌─┴─┐   ┌─┴─┐┌───┐ ░  ║ └╥┘┌─┐      
   q_2: ───────────────────────┤ X ├┤ X ├─X─┤ X ├┤ X ├─░──╫──╫─┤M├──────
        ┌─────────┐┌──────────┐└───┘└───┘ │ └───┘└─┬─┘ ░  ║  ║ └╥┘┌─┐   
   q_3: ┤ Rx(π/4) ├┤ Rx(-π/4) ├───────────┼────────■───░──╫──╫──╫─┤M├───
        └─────────┘└──────────┘           │            ░  ║  ║  ║ └╥┘┌─┐
   q_4: ──────────────────────────────────X────────────░──╫──╫──╫──╫─┤M├
                                                       ░  ║  ║  ║  ║ └╥┘
   c: 5/══════════════════════════════════════════════════╬══╬══╬══╬══╬═
                                                          ║  ║  ║  ║  ║ 
meas: 5/══════════════════════════════════════════════════╩══╩══╩══╩══╩═
                                                          0  1  2  3  4

In [147]:
qi_backend = QI.get_backend('Starmon-5')
qi_job = execute(starmon, backend = qi_backend, shots=1024)
qi_result = qi_job.result()
histogram = qi_result.get_counts(starmon)

In [148]:
def error_measured(regex):
    return (regex[0] == '1' or regex[1] == '1' or regex[3]=='1')

def error_detected(regex):
    return (regex[2] == '1' or regex[4] == '1')

In [149]:
error_count = 0
error_flagged = 0
successful_detection = 0
for i in range(2**5):
    regex = bin(i)[2::].zfill(5)
    if regex in histogram:
        if error_measured(regex):
            error_count = error_count + histogram[regex]
        if error_detected(regex):
            error_flagged = error_flagged+histogram[regex]
        if error_measured(regex) and error_detected(regex):
            successful_detection = successful_detection + histogram[regex]
        if not error_measured(regex) and not error_detected(regex):
            successful_detection = successful_detection + histogram[regex]

In [ ]:
print(1-(error_count/1024), 1-(error_flagged/1024), successful_detection/1024)

We noticed that the error doesn't scale with the depth. We checked the compiled version and discovered that the "white noise" gates were commuted through and cancelled by the transpiler. How to get around this?